# Water bodies detection sub-workflow

Invoke a Common Workflow Language Workflow for detecting water bodies

This notebook is linked to: https://eoap.github.io/zarr-cloud-native-format/sub-workflows/water-bodies-detection/



## Setup

In [ ]:
export WORKSPACE=/workspace/zarr-cloud-native-format
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}

curl -q -L https://github.com/eoap/zarr-cloud-native-format/releases/download/0.5.0/app-water-bodies.0.5.0.cwl > ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl 2> /dev/null

## Run the water bodies detection sub-workflow step

Inspect and use `cwltool` to run the water bodies detection sub-workflow definition:


In [ ]:
cat ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl | yq e '.["$graph"][2]' -

Run the CWL description, but first prepare the parameters.

The previous step generated part of inputs for the water bodies detection sub-workflow:

In [ ]:
cat convert-search-results.json

Let's add the `bands` and `epsg`:

In [ ]:
cat convert-search-results.json | jq .aoi

In [ ]:
aoi=$( cat convert-search-results.json | jq '.aoi' )

echo ${aoi}

In [ ]:
item=$( cat convert-search-results.json | jq '.items[0]' )

echo ${item}

Let's create the parameter file:

In [ ]:
cat <<EOF > water-bodies-params.yaml
item: ${item}
aoi: ${aoi}
bands:
  - green
  - nir
epsg: "4326"
EOF

cat water-bodies-params.yaml | yq .

In [ ]:


cwltool \
    --podman \
    --outdir ${WORKSPACE}/runs \
    ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl#detect_water_body \
    water-bodies-params.yaml > water-bodies-results.json 2> water-bodies.log

Let's look at the content of the stderr:

In [ ]:
cat water-bodies.log | egrep -v "WARNING|JSHINT"

Let's inspect the stdout produced. There are two `output` blocks with the id `aoi` and `items`. 

These are what the water bodies detection sub-workflow expects as inputs

In [ ]:
cat water-bodies-results.json | jq . -